In [1]:
import pandas as pd
import unicodedata
from datetime import date, timedelta
# from sklearn.metrics import mean_absolute_error, mean_squared_error
import logging

logging.basicConfig(level=logging.INFO)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# pd.options.display.float_format = '{:.0f}'.format
pd.options.display.float_format = None

In [2]:
def load_data(file_path):
    try:
        return pd.read_csv(file_path)
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
        return None

def normalize_name(name):
    normalized_name = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8')
    return normalized_name


In [3]:
sp_log = load_data('output/sp_log_2023.csv')
spk_history = load_data('output/spk_history.csv')

# Date manipulation
today = date.today()
yesterday = today - timedelta(days=14)

In [4]:
sp_log['Date'] = pd.to_datetime(sp_log['Date'])
spk_history['commence_time'] = pd.to_datetime(spk_history['commence_time'])

spk_yesterday = spk_history[spk_history['commence_time'].dt.date == yesterday]
sp_log_yesterday = sp_log[sp_log['Date'].dt.date == yesterday]

sp_log_yesterday = sp_log_yesterday[['Name', 'SO', 'IP']]
sp_log_yesterday['Name'] = sp_log_yesterday['Name'].apply(normalize_name)

In [5]:
df = pd.merge(spk_yesterday, sp_log_yesterday, how='left', on='Name')

In [6]:
df_over = df[df['bet_over'] == 1]
df_under = df[df['bet_under'] == 1]
df_over['win'] = df_over['SO'] > df_over['prop_k']
df_under['win'] = df_under['SO'] < df_under['prop_k']

/var/folders/y0/3zjhrtwj00scq7bwsxt8b59w0000gn/T/ipykernel_64596/735911916.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_over['win'] = df_over['SO'] > df_over['prop_k']
/var/folders/y0/3zjhrtwj00scq7bwsxt8b59w0000gn/T/ipykernel_64596/735911916.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_under['win'] = df_under['SO'] < df_under['prop_k']


In [8]:
wager_amount = 100

# def calculate_return_over(row, wager_amount=100):
#     odds = row["over"]
#     win = row["win"]
#     if win:
#         if odds >= 100:
#             return round(odds / 100 * wager_amount, 2)
#         else:
#             return round(wager_amount / abs(odds) * 100, 2)
#     else:
#         return -wager_amount

# def calculate_return_under(row, wager_amount=100):
#     odds = row["under"]
#     win = row["win"]
#     if win:
#         if odds >= 100:
#             return round(odds / 100 * wager_amount, 2)
#         else:
#             return round(wager_amount / abs(odds) * 100, 2)
#     else:
#         return -wager_amount
    
def calculate_return(row, column_name, wager_amount=100):
    odds = row[column_name]
    win = row["win"]
    
    if win:
        if odds >= 100:
            return round(odds / 100 * wager_amount, 2)
        else:
            return round(wager_amount / abs(odds) * 100, 2)
    else:
        return -wager_amount




In [9]:

df_over['wager_return'] = df_over.apply(calculate_return, column_name='over', axis=1)
df_under['wager_return'] = df_under.apply(calculate_return, column_name='under', axis=1)

/var/folders/y0/3zjhrtwj00scq7bwsxt8b59w0000gn/T/ipykernel_64596/4213393201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_over['wager_return'] = df_over.apply(calculate_return, column_name='over', axis=1)
/var/folders/y0/3zjhrtwj00scq7bwsxt8b59w0000gn/T/ipykernel_64596/4213393201.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_under['wager_return'] = df_under.apply(calculate_return, column_name='under', axis=1)


In [10]:
daily_over_return = round(df_over['wager_return'].sum(),2)
daily_over_wager = df_over.shape[0] * wager_amount
daily_over_roi = round(daily_over_return / daily_over_wager * 100, 2)
daily_over_bets = df_over.shape[0]
daily_over_losers = len(df_over[df_over['win'] == True])
daily_over_winners = len(df_over[df_over['win'] == False])

daily_under_return = round(df_under['wager_return'].sum(),2)
daily_under_wager = df_under.shape[0] * wager_amount
daily_under_roi = round(daily_under_return / daily_under_wager * 100, 2)
daily_under_bets = df_under.shape[0]
daily_under_losers = len(df_under[df_under['win'] == True])
daily_under_winners = len(df_under[df_under['win'] == False])


In [12]:
data = {
    "Over Bets": daily_over_bets,
    "Over Win": daily_over_winners,
    "Over Loss": daily_over_losers,
    "Over Return": daily_over_return,
    "Over ROI": daily_over_roi,
    "Under Bets": daily_under_bets,
    "Under Win": daily_under_winners,
    "Under Loss": daily_under_losers,
    "Under Return": daily_under_return,
    "Under ROI": daily_under_roi,
    "Total Bets": daily_over_bets + daily_under_bets,
    "Total Win": daily_over_winners + daily_under_winners,
    "Total Loss": daily_over_losers + daily_under_losers,
    "Total Return": daily_over_return + daily_under_return,
    "Total ROI": round((daily_over_return + daily_under_return) / (daily_over_wager + daily_under_wager) * 100, 2)
}

In [13]:
df_sum = pd.DataFrame(data, index=[yesterday])

In [14]:
df_sum

,Over Bets,Over Win,Over Loss,Over Return,Over ROI,Under Bets,Under Win,Under Loss,Under Return,Under ROI,Total Bets,Total Win,Total Loss,Total Return,Total ROI
2023-07-04,7,5,2,-296.0,-42.29,7,2,5,172.77,24.68,14,7,7,-123.23,-8.8


In [15]:
df_over

,Name,Handedness,Team,Opponent,xK,prop_k,over,over_odds,x_over,over_diff,under,under_odds,x_under,under_diff,bet_over,bet_under,commence_time,SO,IP,win,wager_return
2,Kyle Freeland,L,COL,HOU,4.56,3.5,102.0,0.495,0.668,0.173,-130.0,0.565,0.332,-0.233,1,0,2023-07-04 16:10:00,3.0,6.1,False,-100.0
3,Wade Miley,L,MIL,CHC,4.00,3.5,102.0,0.495,0.566,0.071,-130.0,0.565,0.434,-0.131,1,0,2023-07-04 16:10:00,4.0,5.0,True,102.0
6,Kolby Allard,L,ATL,CLE,4.72,3.5,-124.0,0.554,0.693,0.139,-102.0,0.505,0.307,-0.198,1,0,2023-07-04 19:10:00,3.0,5.0,False,-100.0
12,Kodai Senga,R,NYM,ARI,6.18,5.5,-116.0,0.537,0.583,0.046,-110.0,0.524,0.417,-0.107,1,0,2023-07-04 16:10:00,NaN,NaN,False,-100.0
17,Logan Gilbert,R,SEA,SFG,6.34,5.5,102.0,0.495,0.607,0.112,-128.0,0.561,0.393,-0.168,1,0,2023-07-04 16:35:00,7.0,9.0,True,102.0
19,Shohei Ohtani,R,LAA,SDP,8.12,7.5,-102.0,0.505,0.563,0.058,-124.0,0.554,0.437,-0.117,1,0,2023-07-04 18:40:00,5.0,5.0,False,-100.0
23,Lucas Giolito,R,CHW,TOR,6.18,5.5,-126.0,0.558,0.583,0.025,-102.0,0.505,0.417,-0.088,1,0,2023-07-04 20:10:00,4.0,6.0,False,-100.0


In [16]:
df_under

,Name,Handedness,Team,Opponent,xK,prop_k,over,over_odds,x_over,over_diff,under,under_odds,x_under,under_diff,bet_over,bet_under,commence_time,SO,IP,win,wager_return
0,Patrick Corbin,L,WAS,CIN,3.30,4.5,102.0,0.495,0.237,-0.258,-128.0,0.561,0.763,0.202,0,1,2023-07-04 11:05:00,2.0,5.0,True,78.12
1,Jesus Luzardo,L,MIA,STL,5.96,6.5,-110.0,0.524,0.388,-0.136,-116.0,0.537,0.612,0.075,0,1,2023-07-04 13:10:00,8.0,6.0,False,-100.00
4,JP Sears,L,OAK,DET,5.09,5.5,108.0,0.481,0.399,-0.082,-138.0,0.580,0.601,0.021,0,1,2023-07-04 18:40:00,4.0,7.1,True,72.46
8,Clarke Schmidt,R,NYY,BAL,4.47,4.5,-128.0,0.561,0.462,-0.099,102.0,0.495,0.538,0.043,0,1,2023-07-04 13:05:00,7.0,5.0,False,-100.00
11,Brandon Bielak,R,HOU,COL,3.82,4.5,100.0,0.500,0.336,-0.164,-128.0,0.561,0.664,0.103,0,1,2023-07-04 16:10:00,4.0,7.0,True,78.12
21,Shane Bieber,R,CLE,ATL,4.96,5.5,114.0,0.467,0.377,-0.090,-144.0,0.590,0.623,0.033,0,1,2023-07-04 19:10:00,4.0,4.2,True,69.44
25,Emmet Sheehan,R,LAD,PIT,4.02,5.5,106.0,0.485,0.218,-0.267,-134.0,0.573,0.782,0.209,0,1,2023-07-04 21:10:00,4.0,3.2,True,74.63
